In [ ]:
__author__ = 'Knut Olsen <knut.olsen@noirlab.edu' # single string; emails in <>
__version__ = '20210107' # yyyymmdd; version datestamp of this notebook
__datasets__ = ['delve']  # datasets used in this notebook; for available datasets, see cell "Available datasets in Data Lab" further below
__keywords__ = [''], # keywords relevant to this notebook, e.g. ['science case','dwarf galaxies'] Use only keywords from the master list: https://github.com/noaodatalab/notebooks-latest/blob/master/internal/keywords.txt

# Accessing the DELVE DR1 catalog
*Knut Olsen* and the DELVE Team

### Table of contents
* [Summary](#summary)
* [Disclaimer & Attribution](#attribution)
* [Imports & setup](#import)
* [Accessing the database](#database)
* [Resources and references](#resources)

<a class="anchor" id="summary"></a>

# Summary
This is a brief intro to using the DELVE DR1 catalog through Data Lab.

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution
If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445

* Data Lab disclaimer: http://datalab.noao.edu/disclaimers.php

<a class="anchor" id="import"></a>
# Imports and setup

In [ ]:
# std lib
from getpass import getpass
import os
import time

# 3rd party # remove imports that are not needed
import numpy as np
import pylab as plt
import matplotlib
import healpy as hp
%matplotlib inline

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert

<a class="anchor" id="database"></a>
# Accessing the database

First check that you're logged in.  If not, use the commented code to log in to the server.  Your token is stored on the server, so you will remain logged in unless you explicitly log out.

In [ ]:
ac.whoAmI()
# token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))

### List the columns
You can use the queryClient.schema() method to list the column names of the DELVE object table.  The column descriptions are empty for now.

In [ ]:
# schema and table names
wide_schema = 'delve_dr1'
wide_object = 'objects'

In [ ]:
print(qc.schema(wide_schema))

### A basic query
We'll make a basic query, and return the output as a Pandas dataframe.  You can write your query as SQL (Postgres).

In [ ]:
sql = '''SELECT * FROM {} LIMIT 10'''.format(wide_schema+'.'+wide_object)

print(sql)
df = qc.query(sql=sql,fmt='pandas')

In [ ]:
df

If your query will take longer than the max timeout of 600 seconds, you can run it asynchronously as an ADQL query.  The function below will issue the query and poll for the job to be completed.  If you don't want to wait, you could set the query running and come back later to check its status.

In [ ]:
def longquery(query,fmt='pandas'):
    jobId = qc.query(adql=query, async_=True)
    print(jobId)
    status='RUNNING'
    while status != 'COMPLETED' and status != 'ERROR':
        status = qc.status(jobId=jobId)
        print("status: %s" % status)
        time.sleep(5)
    if status == 'ERROR':
        print("ERROR results....")
        response = qc.error(jobId=jobId)
    else:
        response = convert(qc.results(jobId=jobId), outfmt=fmt)
        
    return response

In [ ]:
adql = '''SELECT TOP 1000 * FROM {}'''.format(wide_schema+'.'+wide_object)

In [ ]:
dfa = longquery(adql,fmt='pandas')

### Another basic query
Here we'll look for objects associated with a specific exposure number.

In [ ]:
%%time
sql = '''SELECT * FROM {} WHERE expnum_g=605691'''.format(wide_schema+'.'+wide_object)
df1 = qc.query(sql=sql,fmt='pandas')

### A cone search
Data Lab uses Q3C indices to speed up cone search queries.  Here's an example:

In [ ]:
ra,dec = df1['ra'][0],df1['dec'][0]

In [ ]:
sql = '''SELECT ra,dec,mag_auto_g,mag_auto_i FROM {2} where Q3C_RADIAL_QUERY(ra,dec,{0},{1},1.2)'''.format(ra,dec,wide_schema+'.'+wide_object)
print(sql)

In [ ]:
%%time
dfr = qc.query(sql=sql,fmt='pandas')

In [ ]:
print(len(dfr))

In [ ]:
plt.figure(figsize=(8,8))
plt.hexbin(dfr['ra'],dfr['dec'])
plt.xlabel('RA')
plt.ylabel('Dec')

In [ ]:
plt.figure(figsize=(8,8))
plt.hexbin(dfr['mag_auto_g']-dfr['mag_auto_i'],dfr['mag_auto_g'],extent=(-1,4,14,24))
plt.xlim(-1,4)
plt.ylim(24,14)
plt.xlabel('g-i')
plt.ylabel('g')

### Make a density map
Here we'll issue a query that aggregates objects by their Healpix ID on an NSIDE=256 RING scheme, and display the object density map.

In [ ]:
%%time
sql = '''SELECT AVG(ra) AS ra0,AVG(dec) AS dec0,COUNT(ra) as nb,ring256 FROM {} GROUP BY ring256'''.format(wide_schema+'.'+wide_object)
df2 = qc.query(sql=sql,fmt='pandas')

In [ ]:
map = np.zeros(hp.nside2npix(256))
map[df2['ring256']] = df2['nb']

In [ ]:
lon=np.arange(3600)/10
lat=np.zeros(3600)
plt.figure(figsize=(8,8))
hp.orthview(map,rot=(180,-30),min=0,max=10000,half_sky=True,fig=1,coord='C',notext=True)
hp.graticule(c='w')
foo=hp.projplot(lon,lat,'ro',coord='G',lonlat=True,markersize=1)

### Star/galaxy separation in DELVE-WIDE
The delve_dr1.objects catalog has several columns, named extended_class[g,r,i,z], that contain a morphological class based on the spread_model parameter.  Objects with extended_class = 0 are likely point sources.

In [ ]:
# select point sources with g-r<0.75, 19.5<g<22, and gerr<0.2
delvequery = """SELECT avg(ra) as ra0,avg(dec) as dec0,hpx2048,count(hpx2048) as nb
           FROM {}
           WHERE 
           AND (mag_auto_g-mag_auto_r)<0.75 AND mag_auto_g>19.5 AND magerr_auto_g<0.2
           AND extended_class_g = 0 AND extended_class_r = 0
           GROUP BY hpx2048
        """.format(wide_schema+'.'+wide_object)
print(delvequery)

In [ ]:
%%time
df_ps = qc.query(sql = delvequery, fmt='pandas', timeout=600)

In [ ]:
map_ps = np.zeros(hp.nside2npix(2048))
map_ps[df_ps['hpx2048']] = df_ps['nb'] # NB: hpx2048 is on the NESTed scheme

In [ ]:
plt.figure(figsize=(10,10))
hp.orthview(map_ps,rot=(180,-30),min=0,max=10,half_sky=True,fig=1,coord='C',notext=True,nest=True)
hp.graticule(c='w')
foo=hp.projplot(lon,lat,'ro',coord='G',lonlat=True,markersize=1)

The map might look smoother if we downgrade the resolution a bit.

In [ ]:
map_ps256 = hp.ud_grade(map_ps,256,order_in='NESTED',order_out='NESTED')

In [ ]:
plt.figure(figsize=(10,10))
hp.orthview(map_ps256,rot=(180,-30),min=0,max=10,half_sky=True,fig=1,coord='C',notext=True,nest=True)
hp.graticule(c='w')
foo=hp.projplot(lon,lat,'ro',coord='G',lonlat=True,markersize=1)

### More info
Have a look at the HowTos and Science Examples folders in the notebooks directory for other things you might want to learn how to do.

<a class="anchor" id="resources"></a>
# Resources and references
